<a href="https://colab.research.google.com/github/Battestin/Veritas.AI/blob/main/VeritasAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip -q install google-genai

# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [2]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7

In [3]:
def agente_coletor_de_evidencias(topico, data_de_hoje):
    coletor = Agent(
        name="agente_coletor_de_evidencias",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um verificador de fatos que usa a ferramenta google_search para encontrar fontes confiáveis, como:
        - Reportagens de grandes veículos jornalísticos
        - Artigos científicos
        - Sites de checagem de fatos (e.g., Aos Fatos, Lupa, Snopes, FactCheck.org)
        Busque refutações ou confirmações sobre o tópico fornecido. Ignore opiniões de redes sociais ou fóruns.
        Traga trechos relevantes e URLs de até 5 fontes distintas, priorizando atualidade (últimos 12 meses).
        """,
        description="Busca fontes confiáveis sobre um tema polêmico",
        tools=[google_search]
    )

    entrada = f"Tópico: {topico}\nData: {data_de_hoje}"
    evidencias = call_agent(coletor, entrada)
    return evidencias


In [20]:
def agente_avaliador_de_veracidade(topico, evidencias):
    avaliador = Agent(
        name="agente_avaliador_de_veracidade",
        model="gemini-2.5-pro-preview-03-25",
        instruction=f"""
        Você é um avaliador crítico. Receberá fontes jornalísticas e científicas sobre um tópico controverso.
        Sua função:
        1. Verificar se o consenso das fontes aponta para verdade, falsidade, imprecisão ou incerteza.
        2. Emitir um parecer objetivo (uma dessas 4 opções): VERDADEIRO, FALSO, IMPRECISO, NÃO CONCLUSIVO.
        3. Justificar sua classificação com base nas evidências apresentadas.
        Evite linguagem vaga. Seja direto e técnico.
        """,
        description="Avalia veracidade com base nas evidências."
    )

    entrada = f"Tópico: {topico}\nEvidências:\n{evidencias}"
    parecer = call_agent(avaliador, entrada)
    return parecer


In [19]:
def agente_explicador(topico, parecer):
    explicador = Agent(
        name="agente_explicador",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
        Você é um educador popular. Receberá uma análise técnica sobre um tema polêmico.
        Sua missão: explicar essa análise de forma clara e acessível, como se estivesse explicando para alguém curioso, mas leigo.
        Use analogias, exemplos, e um tom respeitoso e levemente irônico, se couber.
        Evite jargões técnicos.
        Finalize com uma dica de pensamento crítico ou leitura adicional.
        Você diz o necessário e não se extende muito em suas explicações.
        Você utiliza de formatações e elementos visuais para ajudar a legibilidade.
        """,
        description="Explica de forma acessível o parecer técnico."
    )

    entrada = f"Tópico: {topico}\nParecer técnico:\n{parecer}"
    explicacao = call_agent(explicador, entrada)
    return explicacao


In [18]:
topico = input("❓ Digite a afirmação que você quer verificar (ex: 'A Terra é plana'): ")
data_de_hoje = date.today().strftime("%d/%m/%Y")

if not topico:
    print("Você esqueceu de digitar a afirmação.")
else:
    print(f"🔎 Verificando: {topico}")

    evidencias = agente_coletor_de_evidencias(topico, data_de_hoje)
    #display(to_markdown(evidencias))

    parecer = agente_avaliador_de_veracidade(topico, evidencias)
    #display(to_markdown(parecer))

    explicacao = agente_explicador(topico, parecer)
    display(to_markdown(explicacao))


❓ Digite a afirmação que você quer verificar (ex: 'A Terra é plana'): virginia joga no tigrinho
🔎 Verificando: virginia joga no tigrinho


> ## Virginia e o Tigrinho: o que diz a análise? 🐅
> 
> E aí, pessoal! 👀 Vamos desmistificar essa história da Virginia e o tal "tigrinho"?
> 
> ### O que rolou? 🤔
> 
> *   Aparentemente, a galera anda dizendo que a Virginia Fonseca "joga no tigrinho".
> *   Mas será que é isso mesmo? 🤔
> 
> ### O que a análise técnica diz? 🤓
> 
> **Calma lá!** A análise foi clara: dizer que a Virginia "joga no tigrinho" é **IMPRECISO**.
> 
> *   **Ela faz propaganda:** Sim, a Virginia fez publicidade para plataformas de apostas que oferecem o tal jogo do tigrinho.
> *   **Ela joga?** Não há provas de que ela *realmente* jogue no tigrinho. As fontes dizem que ela joga na Mega-Sena, por exemplo.
> *   **Ela lucra com a derrota dos outros?** Ela nega essa história e diz que sempre alertou sobre os riscos dos jogos.
> 
> ### Em resumo: ✅ ou ❌?
> 
> *   ✅ Ela divulga o jogo.
> *   ❌ Não há provas de que ela joga.
> 
> ### Moral da história 📖
> 
> Antes de sair espalhando por aí que a Virginia é jogadora assídua do tigrinho, vale a pena conferir os fatos. 😉
> 
> **Dica extra:** desconfie de tudo que você vê na internet e sempre busque informações em fontes confiáveis! 😉
> 
